In [1]:
import keras
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.losses import categorical_crossentropy
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.metrics import top_k_categorical_accuracy
import sklearn
import warnings

C:\Users\VuDinhPhong\Anaconda3\envs\YOLO-env\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)


In [3]:
TRAINING_IMG_FOLDER = os.path.join(os.getcwd(), 'train')

RESIZE_WIDTH = 178
RESIZE_HEIGH = 178

INPUT_WIDTH = 178
INPUT_HEIGHT = 178

BATCH_SIZE = 32
EPOCH = 100

FIG_SIZE = 15

In [4]:
train_csv = pd.read_csv('train.csv')
train_csv.head()

,Image,Id
0,00022e1a.jpg,w_e15442c
1,000466c4.jpg,w_1287fbc
2,00087b01.jpg,w_da2efe0
3,001296d5.jpg,w_19e5482
4,0014cfdf.jpg,w_f22f3e3


In [5]:
def load_imgs_from_folder(folder, filenames=None):
    """
        Read images into a numpy array
        Input: 
            - folder: a path
            - filenames: list of images name in folder 
    """
    imgs = []
    file_names = filenames if filenames is not None else train_csv.Image
    for f in file_names:
        img = cv2.imread(os.path.join(folder, f))
        img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGH))
        img = img.reshape(1, RESIZE_WIDTH, RESIZE_HEIGH, 3)
        imgs.append(img)
    imgs = np.vstack(imgs)
    return imgs

def load_data_from_file(hdf5_file):
    """
        Load numpy array from hdf5
        
    """
    raise NotImplementedError
    

In [6]:
def plot_imgs(imgs_list, imgs_name=None):
    
    fig = plt.figure(figsize=(FIG_SIZE,FIG_SIZE))
    if imgs_name is None:
        imgs_name = np.arange(len(imgs_list))
        
    _COL = 5
    _ROW = (len(imgs_list)/_COL) + 1
    
    if not isinstance(imgs_list, list):
        imgs_list = [imgs_list]
    
    for i,im in enumerate(imgs_list):
        
        ax = fig.add_subplot(_ROW, _COL, i+1)
        
        ax.set_title(imgs_name[i])
        ax.imshow(im, cmap='gray')

In [7]:
class LabelOntHotEncoder():
    """
         A class which transform labels data to ont hot encoding
    """
    
    def __init__(self, Y):
        """
            Initialize with training label Y
        """
        if not any(Y):
            raise ValueError('Intialize error, missing Y')
        self.label_encoder = LabelEncoder()
        self.label_binarizer = LabelBinarizer()
        self.Y_encoded = self.label_encoder.fit_transform(Y)
        self.y_one_hot = self.label_binarizer.fit_transform(self.Y_encoded)
    
    def Y_one_hot(self):
        return self.y_one_hot
    
    def encode(self, y):
        """
            Input y: array of shape (N, ) contains category of class like 'new_whale', ...
            return: One hot like (N, num_class)
        """
        y_encoded = self.label_encoder.transform(y)
        return self.label_binarizer.transform(y_encoded)
    
    def decode(self, y):
        """
            Input y: numpy array of shape (N, num_class) contains ont not like [ [0,1,0,0,...], [1,0,0,0,...] ]
            return: numpy array of string classes
        """
        y_decoded = self.label_binarizer.inverse_transform(y)
        return self.label_encoder.inverse_transform(y_decoded)
    
    def num_class(self):
        
        return len(self.label_binarizer.classes_)

In [8]:
x = load_imgs_from_folder(TRAINING_IMG_FOLDER, train_csv.Image)
x.shape

(9850, 178, 178, 3)

In [9]:
label_encoder = LabelOntHotEncoder(train_csv.Id.values)
y_one_hot = label_encoder.Y_one_hot()
y_one_hot.shape

(9850, 4251)

In [10]:
train_datagen = ImageDataGenerator(featurewise_center=True,  
                                   featurewise_std_normalization=True, 
                                   rotation_range=20, rescale=1./255, 
                                   zoom_range=0.2, 
                                   validation_split = 0.2)
train_datagen.fit(x)


In [11]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(axis=-1)(x)
        x = layers[i](x)

    new_model = Model(inputs=layers[0].input, outputs=x)
    return new_model

In [12]:
vgg_net = VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(INPUT_WIDTH,INPUT_HEIGHT,3))

# Adding fully-connected at the end
flatten = Flatten()(vgg_net.output)
fully = Dense(4096, activation='relu')(flatten)
fully = Dense(4096, activation='relu')(fully)
preds = Dense(label_encoder.num_class(), activation='softmax')(fully)
vgg_net = Model(vgg_net.input, preds)

# Adding BatchNorm
vgg_net = insert_intermediate_layer_in_keras(vgg_net, 3, BatchNormalization)
vgg_net = insert_intermediate_layer_in_keras(vgg_net, 6, BatchNormalization)
vgg_net = insert_intermediate_layer_in_keras(vgg_net, 10, BatchNormalization)
vgg_net = insert_intermediate_layer_in_keras(vgg_net, 14, BatchNormalization)
vgg_net = insert_intermediate_layer_in_keras(vgg_net, 18, BatchNormalization)


vgg_net.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 178, 178, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 178, 178, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 178, 178, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 178, 178, 64)      256       
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 89, 89, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 89, 89, 128)       73856     
_________________________________________________________________
batch_normalization_2 (Batch (None, 89, 89, 128)       512       
__________

In [14]:
def top_5_categorical_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5) 

In [16]:
vgg_net.compile(optimizer=Adam(), 
                loss=categorical_crossentropy, 
                metrics=[top_5_categorical_accuracy], )

In [ ]:
vgg_net.fit_generator(generator=train_datagen.flow(x, y_one_hot, batch_size=BATCH_SIZE),
                      steps_per_epoch=x.shape[0]/BATCH_SIZE, epochs=EPOCH)

Epoch 1/100
308/307 [==============================] - 119s 386ms/step - loss: 8.1496 - top_5_categorical_accuracy: 0.0882
Epoch 2/100
308/307 [==============================] - 113s 365ms/step - loss: 7.7881 - top_5_categorical_accuracy: 0.0925
Epoch 3/100
308/307 [==============================] - 113s 366ms/step - loss: 7.7428 - top_5_categorical_accuracy: 0.0922
Epoch 4/100
308/307 [==============================] - 112s 363ms/step - loss: 7.7314 - top_5_categorical_accuracy: 0.0926
Epoch 5/100
308/307 [==============================] - 113s 366ms/step - loss: 7.7257 - top_5_categorical_accuracy: 0.0923
Epoch 6/100
308/307 [==============================] - 112s 363ms/step - loss: 7.7227 - top_5_categorical_accuracy: 0.0927
Epoch 7/100
308/307 [==============================] - 111s 361ms/step - loss: 7.7210 - top_5_categorical_accuracy: 0.0928
Epoch 8/100
308/307 [==============================] - 112s 364ms/step - loss: 7.7183 - top_5_categorical_accuracy: 0.0925
Epoch 9/100
308/